# Random Silicon Sampling
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for reproducing LLM-based experiments performed in the recent paper <a href="https://arxiv.org/pdf/2402.18144.pdf">Random Silicon Sampling: Simulating Human Sub-Population Opinion Using a Large Language Model Based on Group-Level Demographic Information</a> by Seungjong Sun, Eungu Lee, Dongyan Nan, Xiangying Zhao, Wonbyung Lee, Bernard J. Jansen and Jang Hyun Kim. 

We focus on the variables and questions shown in pp.13-14 of the paper, reproduced at the end of this notebook, and also include a method for sense checking that the LLMs understood the prompts (e.g., we can ask the agents to describe current events in order to verify comprehension of a prompt that noted "Today is November 3, 2020.").

EDSL is an open-source library for simulating surveys and experiements with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

In [1]:
# ! pip install edsl

## Designing AI agents
We start by designing personas for AI agents that will respond to the survey. This is done by creating `Agent` objects and passing them dictionaries of desired traits:

In [2]:
from edsl import Agent, AgentList

We can inspect an `Agent` object with an example:

In [3]:
Agent.example()

Agent(traits = {'age': 22, 'hair': 'brown', 'height': 5.5})

### Creating personas
We will create agents with randomly selected combinations of demographics, so we start by listing the possible values for each trait that we want to use:

In [4]:
races = ["white", "black", "asian", "native American", "hispanic"]
genders = ["man", "woman"]
ages = range(21, 90)
ideologies = [
    "extremely liberal",
    "liberal",
    "slightly liberal",
    "moderate",
    "slightly conservative",
    "conservative",
    "extremely conservative",
]
parties = [
    "a strong Democrat",
    "a weak Democrat",
    "an independent who leans Democratic",
    "an independent",
    "an independent who leans Republican",
    "a weak Republican",
    "a strong Republican",
]
political_interests = ["very", "somewhat", "not very", "not at all"]
church_attendances = ["attend church", "do not attend church"]
political_discussions = [
    "I like to discuss politics with my family and friends.",
    "I never discuss politics with my family or friends.",
]

### Base persona
For convenience in analyzing the results data later on, we create both a narrative persona mirroring the prompt in the paper and also include a dictionary of the relevant demographic values so that we can access them directly (e.g., in order to be able to query <i>age=30</i> as well as <i>"I am 30 years old."</i>).

In [5]:
agent_traits = [
    {
        "context": "Today is November 3, 2020.",
        "persona": f"""Racially I am {race}. I am a {gender}. I am {age} years old. I am {ideology}. I am a {party}. I am {political_interest} in politics. I {church_attendance}. I {political_discussion}.""",
        "race": race,
        "gender": gender,
        "age": age,
        "ideology": ideology,
        "party": party,
        "political_interest": political_interest,
        "church_attendance": church_attendance,
        "political_discussion": political_discussion,
    }
    for race in races
    for gender in genders
    for age in ages
    for ideology in ideologies
    for party in parties
    for political_interest in political_interests
    for church_attendance in church_attendances
    for political_discussion in political_discussions
]

Agents can also be passed a name; here we simply number them:

In [6]:
agent_list = AgentList(
    [Agent(traits=traits, name=index) for index, traits in enumerate(agent_traits)]
)

In [7]:
len(agent_list)

540960

In [8]:
import random

sample_agents = random.sample(agent_list, 5)

In [9]:
sample_agents

[Agent(name = 82471, traits = {'context': 'Today is November 3, 2020.', 'persona': 'Racially I am white. I am a woman. I am 57 years old. I am liberal. I am a an independent who leans Democratic. I am somewhat in politics. I do not attend church. I I never discuss politics with my family or friends..', 'race': 'white', 'gender': 'woman', 'age': 57, 'ideology': 'liberal', 'party': 'an independent who leans Democratic', 'political_interest': 'somewhat', 'church_attendance': 'do not attend church', 'political_discussion': 'I never discuss politics with my family or friends.'}),
 Agent(name = 30066, traits = {'context': 'Today is November 3, 2020.', 'persona': 'Racially I am white. I am a man. I am 59 years old. I am slightly liberal. I am a an independent. I am very in politics. I do not attend church. I I like to discuss politics with my family and friends..', 'race': 'white', 'gender': 'man', 'age': 59, 'ideology': 'slightly liberal', 'party': 'an independent', 'political_interest': 've

## Creating survey questions
Next we create `Question` objects for the questions that we want to ask the agents. EDSL comes with a variety of question types that we can use based on the form of the response that we want (e.g., a selection from a list of options, or unstructured text). See [examples of all question types](https://docs.expectedparrot.com/en/latest/questions.html).

Here we replicate the questions in pp.13-14 using `QuestionMultipleChoice`. Note that we included the context "Today is November 3, 2020" in the agent traits above in order to mirror the prompts in the paper but we could also include it as a part of each `question_text`:

In [10]:
from edsl.questions import QuestionMultipleChoice

In [11]:
q_election = QuestionMultipleChoice(
    question_name="election",
    question_text="""In the 2020 presidential election, Donald Trump is the Republican candidate,
    and Joe Biden is the Democratic candidate, and I voted for:""",
    question_options=["Donald Trump", "Joe Biden"],
)

q_race_diversity = QuestionMultipleChoice(
    question_name="race_diversity",
    question_text="""Does the increasing number of people of many different races and
    ethnic groups in the United States make this country a better
    place to live, a worse place to live, or does it make no difference?""",
    question_options=["Better", "Worse", "Makes no difference"],
)

q_gender_role = QuestionMultipleChoice(
    question_name="gender_role",
    question_text="""Do you think it is better, worse, or makes no difference for the
    family as a whole if the man works outside the home and the
    woman takes care of the home and family?""",
    question_options=["Better", "Worse", "Makes no difference"],
)

q_current_economy = QuestionMultipleChoice(
    question_name="current_economy",
    question_text="""What do you think about the state of the economy these days in
    the United States?""",
    question_options=["Very good", "Good", "Neither good nor bad", "Bad", "Very bad"],
)

q_drug_addiction = QuestionMultipleChoice(
    question_name="drug_addiction",
    question_text="""Do you think the federal government should be doing more about
    the opioid drug addiction issue, should be doing less, or is it
    currently doing the right amount?""",
    question_options=[
        "Should be doing more",
        "Should be doing less",
        "Is doing the right amount",
    ],
)

q_climate_change = QuestionMultipleChoice(
    question_name="climate_change",
    question_text="""How much, if at all, do you think climate change is currently
    affecting severe weather events or temperature patterns in the
    United States?""",
    question_options=[
        "Not at all",
        "A little",
        "A moderate amount",
        "A lot",
        "A great deal",
    ],
)

q_gay_marriage = QuestionMultipleChoice(
    question_name="gay_marriage",
    question_text="""Which comes closest to your view?""",
    question_options=[
        "Gay and lesbian couples should be allowed to legally marry.",
        "Gay and lesbian couples should be allowed to form civil unions but not legally marry.",
        "There should be no legal recognition of gay or lesbian couples’ relationship.",
    ],
)

q_refugee_allowing = QuestionMultipleChoice(
    question_name="refugee_allowing",
    question_text="""Do you favor, oppose, or neither favor nor oppose allowing
    refugees who are fleeing war, persecution, or natural disasters
    in other countries to come to live in the United States?""",
    question_options=["Favor", "Oppose", "Neither favor nor oppose"],
)

q_health_insurance = QuestionMultipleChoice(
    question_name="health_insurance",
    question_text="""Do you favor an increase, decrease, or no change in government
    spending to help people pay for health insurance when people
    cannot pay for it all themselves?""",
    question_options=["Increase", "Decrease", "No change"],
)

q_gun_regulation = QuestionMultipleChoice(
    question_name="gun_regulation",
    question_text="""Do you think the federal government should make it more
    difficult for people to buy a gun than it is now, make it easier
    for people to buy a gun, or keep these rules about the same
    as they are now?""",
    question_options=["More difficult", "Easier", "Keep these rules about the same"],
)

q_income_inequality = QuestionMultipleChoice(
    question_name="income_inequality",
    question_text="""Do you favor, oppose, or neither favor nor oppose the
    government trying to reduce the difference in incomes between
    the richest and poorest households?""",
    question_options=["Favor", "Oppose", "Neither favor nor oppose"],
)

Next we combine the questions into a `Survey` object which takes the list of question ids:

In [12]:
from edsl import Survey

survey = Survey(
    questions=[
        q_election,
        q_race_diversity,
        q_gender_role,
        q_current_economy,
        q_drug_addiction,
        q_climate_change,
        q_gay_marriage,
        q_refugee_allowing,
        q_health_insurance,
        q_gun_regulation,
        q_income_inequality,
    ]
)

## Selecting language models
We can specify the LLMs that we want to use in simulating results with the `Model` class. EDSL works with many popular models; we can check a current list of them:

In [13]:
from edsl import Model

Model.available()

[['01-ai/Yi-34B-Chat', 'deep_infra', 0],
 ['81549361/waifu70b-llama2', 'deep_infra', 1],
 ['Austism/chronos-hermes-13b-v2', 'deep_infra', 2],
 ['Gryphe/MythoMax-L2-13b', 'deep_infra', 3],
 ['Gryphe/MythoMax-L2-13b-turbo', 'deep_infra', 4],
 ['HuggingFaceH4/zephyr-orpo-141b-A35b-v0.1', 'deep_infra', 5],
 ['Phind/Phind-CodeLlama-34B-v2', 'deep_infra', 6],
 ['bigcode/starcoder2-15b', 'deep_infra', 7],
 ['bigcode/starcoder2-15b-instruct-v0.1', 'deep_infra', 8],
 ['claude-3-haiku-20240307', 'anthropic', 9],
 ['claude-3-opus-20240229', 'anthropic', 10],
 ['claude-3-sonnet-20240229', 'anthropic', 11],
 ['codellama/CodeLlama-34b-Instruct-hf', 'deep_infra', 12],
 ['codellama/CodeLlama-70b-Instruct-hf', 'deep_infra', 13],
 ['cognitivecomputations/dolphin-2.6-mixtral-8x7b', 'deep_infra', 14],
 ['databricks/dbrx-instruct', 'deep_infra', 15],
 ['deepinfra/airoboros-70b', 'deep_infra', 16],
 ['gemini-pro', 'google', 17],
 ['google/codegemma-7b-it', 'deep_infra', 18],
 ['google/gemma-1.1-7b-it', 'dee

Here we select a couple to use in running the survey:

In [14]:
models = [Model(m) for m in ("gpt-3.5-turbo", "gpt-4-1106-preview")]

## Simulating results
We simulate responses to our question by appending the agents to the survey with the `.by()` method, and then append the `.run()` method:

In [15]:
results = survey.by(sample_agents).by(models).run()

## Analyzing results

The `Results` object that is automatically generated included a number of fields about the models and prompts as well as the agent traits and answers. The `edsl` package includes a number of built-in methods for analyzing results with dataframes, SQL and visualization methods:

In [16]:
results.columns

['agent.age',
 'agent.agent_instruction',
 'agent.agent_name',
 'agent.church_attendance',
 'agent.context',
 'agent.gender',
 'agent.ideology',
 'agent.party',
 'agent.persona',
 'agent.political_discussion',
 'agent.political_interest',
 'agent.race',
 'answer.climate_change',
 'answer.current_economy',
 'answer.drug_addiction',
 'answer.election',
 'answer.gay_marriage',
 'answer.gender_role',
 'answer.gun_regulation',
 'answer.health_insurance',
 'answer.income_inequality',
 'answer.race_diversity',
 'answer.refugee_allowing',
 'comment.climate_change_comment',
 'comment.current_economy_comment',
 'comment.drug_addiction_comment',
 'comment.election_comment',
 'comment.gay_marriage_comment',
 'comment.gender_role_comment',
 'comment.gun_regulation_comment',
 'comment.health_insurance_comment',
 'comment.income_inequality_comment',
 'comment.race_diversity_comment',
 'comment.refugee_allowing_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.ma

We can filter results by appending `.filter()` with desired logic and then `.select()` fields to that we want to `.print()`:

In [17]:
(
    results.filter("model.model=='gpt-4-1106-preview'")
    .select(
        "persona",
        "election",
        "race_diversity",
        "gender_role",
        "current_economy",
        "drug_addiction",
        "climate_change",
    )  # , "gay_marriage", "refugee_allowing", "health_insurance", "gun_regulation", "income_inequality")
    .print(format="rich")
)

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ agent         ┃ answer       ┃ answer         ┃ answer        ┃ answer         ┃ answer        ┃ answer         ┃
┃ .persona      ┃ .election    ┃ .race_diversi… ┃ .gender_role  ┃ .current_econ… ┃ .drug_addict… ┃ .climate_chan… ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ Racially I am │ Donald Trump │ Better         │ Makes no      │ Bad            │ Should be     │ A moderate     │
│ hispanic. I   │              │                │ difference    │                │ doing more    │ amount         │
│ am a woman. I │              │                │               │                │               │                │
│ am 59 years   │              │                │               │                │               │                │
│ old. I am     │              │                │               │                │               │                │
│ conservative. │              │                │               │                │               │                │
│ I am a a      │              │                │               │                │               │                │
│ strong        │              │                │               │                │               │                │
│ Republican. I │              │                │               │                │               │                │
│ am not very   │              │                │               │                │               │                │
│ in politics.  │              │                │               │                │               │                │
│ I do not      │              │                │               │                │               │                │
│ attend        │              │                │               │                │               │                │
│ church. I I   │              │                │               │                │               │                │
│ never discuss │              │                │               │                │               │                │
│ politics with │              │                │               │                │               │                │
│ my family or  │              │                │               │                │               │                │
│ friends..     │              │                │               │                │               │                │
├───────────────┼──────────────┼────────────────┼───────────────┼────────────────┼───────────────┼────────────────┤
│ Racially I am │ Joe Biden    │ Better         │ Makes no      │ Bad            │ Should be     │ A lot          │
│ asian. I am a │              │                │ difference    │                │ doing more    │                │
│ woman. I am   │              │                │               │                │               │                │
│ 33 years old. │              │                │               │                │               │                │
│ I am slightly │              │                │               │                │               │                │
│ conservative. │              │                │               │                │               │                │
│ I am a a weak │              │                │               │                │               │                │
│ Democrat. I   │              │                │               │                │               │                │
│ am not at all │              │                │               │                │               │                │
│ in politics.  │              │                │               │                │               │                │
│ I attend      │              │                │               │                │               │                │
│ church. I I   │              │                │       

We can add any desired labels to our tables:

In [18]:
(
    results.filter("model.model=='gpt-4-1106-preview'")
    .select("age", "gender", "climate_change", "current_economy", "income_inequality")
    .print(
        pretty_labels={
            "agent.age": "Age",
            "agent.gender": "Gender",
            "answer.climate_change": q_climate_change.question_text.replace("\n  ", ""),
            "answer.current_economy": q_current_economy.question_text.replace(
                "\n  ", ""
            ),
            "answer.income_inequality": q_income_inequality.question_text.replace(
                "\n  ", ""
            ),
        },
        format="rich",
    )
)

┏━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃     ┃        ┃ How much, if at all, do you    ┃                                ┃ Do you favor, oppose, or       ┃
┃     ┃        ┃ think climate change is        ┃                                ┃ neither favor nor oppose the   ┃
┃     ┃        ┃ currently  affecting severe    ┃                                ┃ government trying to reduce    ┃
┃     ┃        ┃ weather events or temperature  ┃ What do you think about the    ┃ the difference in incomes      ┃
┃     ┃        ┃ patterns in the  United        ┃ state of the economy these     ┃ between  the richest and       ┃
┃ Age ┃ Gender ┃ States?                        ┃ days in  the United States?    ┃ poorest households?            ┃
┡━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 59  │ woman  │ A moderate amount              │ Bad                            │ Oppose                         │
├─────┼────────┼────────────────────────────────┼────────────────────────────────┼────────────────────────────────┤
│ 33  │ woman  │ A lot                          │ Bad                            │ Neither favor nor oppose       │
├─────┼────────┼────────────────────────────────┼────────────────────────────────┼────────────────────────────────┤
│ 28  │ woman  │ A lot                          │ Bad                            │ Favor                          │
├─────┼────────┼────────────────────────────────┼────────────────────────────────┼────────────────────────────────┤
│ 59  │ man    │ A lot                          │ Bad                            │ Favor                          │
├─────┼────────┼────────────────────────────────┼────────────────────────────────┼────────────────────────────────┤
│ 57  │ woman  │ A great deal                   │ Bad                            │ Favor                          │
└─────┴────────┴────────────────────────────────┴────────────────────────────────┴────────────────────────────────┘

In [19]:
results.sql("select * from self", shape="wide")

,age,agent_instruction,agent_name,church_attendance,climate_change,climate_change_comment,climate_change_question_options,climate_change_question_text,climate_change_question_type,climate_change_raw_model_response,...,refugee_allowing_comment,refugee_allowing_question_options,refugee_allowing_question_text,refugee_allowing_question_type,refugee_allowing_raw_model_response,refugee_allowing_system_prompt,refugee_allowing_user_prompt,temperature,top_logprobs,top_p
0,57,You are answering questions as if you were a h...,82471,do not attend church,A great deal,Climate change is having a great deal of impac...,"['Not at all', 'A little', 'A moderate amount'...","How much, if at all, do you think climate chan...",multiple_choice,{'id': 'chatcmpl-9QCfc3o82PWNwUEMpd14yGhwz1eDE...,...,"I favor allowing refugees who are fleeing war,...","['Favor', 'Oppose', 'Neither favor nor oppose']","Do you favor, oppose, or neither favor nor opp...",multiple_choice,{'id': 'chatcmpl-9QCfcwp7gyPJ0QlHkfKLWJTRezbmW...,You are answering questions as if you were a h...,You are being asked the following question: Do...,0.5,3,1
1,59,You are answering questions as if you were a h...,30066,do not attend church,A great deal,Climate change is having a great deal of impac...,"['Not at all', 'A little', 'A moderate amount'...","How much, if at all, do you think climate chan...",multiple_choice,{'id': 'chatcmpl-9QCfcrJXkyqfWjtqu955YbJfqH5cr...,...,"I favor allowing refugees who are fleeing war,...","['Favor', 'Oppose', 'Neither favor nor oppose']","Do you favor, oppose, or neither favor nor opp...",multiple_choice,{'id': 'chatcmpl-9QCfceeh2Znks7onWlGHO0OfU0905...,You are answering questions as if you were a h...,You are being asked the following question: Do...,0.5,3,1
2,59,You are answering questions as if you were a h...,517323,do not attend church,A little,I believe that climate change may have a small...,"['Not at all', 'A little', 'A moderate amount'...","How much, if at all, do you think climate chan...",multiple_choice,{'id': 'chatcmpl-9QCfcEdi1lC6jhdsYYvN3u39KM0wv...,...,"I favor allowing refugees who are fleeing war,...","['Favor', 'Oppose', 'Neither favor nor oppose']","Do you favor, oppose, or neither favor nor opp...",multiple_choice,{'id': 'chatcmpl-9QCfcHMXfab3ag1idZTBmh4QxDdGW...,You are answering questions as if you were a h...,You are being asked the following question: Do...,0.5,3,1
3,33,You are answering questions as if you were a h...,280365,attend church,A lot,I believe that climate change is currently aff...,"['Not at all', 'A little', 'A moderate amount'...","How much, if at all, do you think climate chan...",multiple_choice,{'id': 'chatcmpl-9QCfcAQqO57dTnw4bOIOywPRZfOWg...,...,I neither favor nor oppose allowing refugees w...,"['Favor', 'Oppose', 'Neither favor nor oppose']","Do you favor, oppose, or neither favor nor opp...",multiple_choice,{'id': 'chatcmpl-9QCfc2lScz0O5ymiUYpj4OlbO071z...,You are answering questions as if you were a h...,You are being asked the following question: Do...,0.5,3,1
4,28,You are answering questions as if you were a h...,276123,do not attend church,A great deal,I believe that climate change is currently aff...,"['Not at all', 'A little', 'A moderate amount'...","How much, if at all, do you think climate chan...",multiple_choice,{'id': 'chatcmpl-9QCfch1sB4kR9Fe7hXkUC9GTKSXsE...,...,"I favor allowing refugees who are fleeing war,...","['Favor', 'Oppose', 'Neither favor nor oppose']","Do you favor, oppose, or neither favor nor opp...",multiple_choice,{'id': 'chatcmpl-9QCfcCW1zymFGGzJMRU6mrqDuDioA...,You are answering questions as if you were a h...,You are being asked the following question: Do...,0.5,3,1
5,59,You are answering questions as if you were a h...,517323,do not attend church,A moderate amount,I believe that climate change may be contribut...,"['Not at all', 'A little', 'A moderate amount'...","How much, if at all, do you think climate chan...",multiple_choice,{'id': 'chatcmpl-9QCfchTHBhTS2kubSXjuRyRMohtW

In [20]:
results.to_pandas().to_csv("results.csv")

In [21]:
results.to_pandas()

,agent.age,agent.agent_instruction,agent.agent_name,agent.church_attendance,agent.context,agent.gender,agent.ideology,agent.party,agent.persona,agent.political_discussion,...,raw_model_response.election_raw_model_response,raw_model_response.gay_marriage_raw_model_response,raw_model_response.gender_role_raw_model_response,raw_model_response.gun_regulation_raw_model_response,raw_model_response.health_insurance_raw_model_response,raw_model_response.income_inequality_raw_model_response,raw_model_response.race_diversity_raw_model_response,raw_model_response.refugee_allowing_raw_model_response,scenario.edsl_class_name,scenario.edsl_version
0,57,You are answering questions as if you were a h...,82471,do not attend church,"Today is November 3, 2020.",woman,liberal,an independent who leans Democratic,Racially I am white. I am a woman. I am 57 yea...,I never discuss politics with my family or fri...,...,{'id': 'chatcmpl-9QCfc3Ocho4qNIwHGvlQkrfDJmsjH...,{'id': 'chatcmpl-9QCfc71sFA3DloXBPtoMhxjaDGHpb...,{'id': 'chatcmpl-9QCfczhn8ji5Zh9zkA5K4VyRUydap...,{'id': 'chatcmpl-9QCfcdt1xAVv9lqbU1uPL38fFErLR...,{'id': 'chatcmpl-9QCfcXWYNzzUTUlVhdc8wyymDgrVU...,{'id': 'chatcmpl-9QCfcu0M0vi7tFwoYD9JB4ZW9DNCj...,{'id': 'chatcmpl-9QCfcLdewEcIgzOIT5jvE1BuDzC34...,{'id': 'chatcmpl-9QCfcwp7gyPJ0QlHkfKLWJTRezbmW...,Scenario,0.1.23
1,59,You are answering questions as if you were a h...,30066,do not attend church,"Today is November 3, 2020.",man,slightly liberal,an independent,Racially I am white. I am a man. I am 59 years...,I like to discuss politics with my family and ...,...,{'id': 'chatcmpl-9QCfcloBHFdbH77XEmWVRuFTJt2HP...,{'id': 'chatcmpl-9QCfc8ylbgJkXeLQArMyn4b9Sf1bo...,{'id': 'chatcmpl-9QCfcxW4oDnGBN8tNIw7bydsH1o4r...,{'id': 'chatcmpl-9QCfc5AfwJF04YT03pILwty9KAEun...,{'id': 'chatcmpl-9QCfc62yaxAAYFA8g9qjJfPXzcMrK...,{'id': 'chatcmpl-9QCfcqg5sysan6ayyRNz4ChhfQ9SK...,{'id': 'chatcmpl-9QCfcZZIGmVBmbIh3hNO1eztCbmkt...,{'id': 'chatcmpl-9QCfceeh2Znks7onWlGHO0OfU0905...,Scenario,0.1.23
2,59,You are answering questions as if you were a h...,517323,do not attend church,"Today is November 3, 2020.",woman,conservative,a strong Republican,Racially I am hispanic. I am a woman. I am 59 ...,I never discuss politics with my family or fri...,...,{'id': 'chatcmpl-9QCfc5mtgE7oyfRnDJZu5jeffLOF8...,{'id': 'chatcmpl-9QCfcKcX6AsbTR20dKh8hji0nTFLP...,{'id': 'chatcmpl-9QCfcqWTYIfDtgEnW7O1i9vnKkrH9...,{'id': 'chatcmpl-9QCfcSYr4gGU1qfVRkqy0MB0Df36p...,{'id': 'chatcmpl-9QCfcPRSUMC7Cn54Zo3OXfAgKgrLn...,{'id': 'chatcmpl-9QCfcpHKKlqb8Pz7lZ5L7Lu2dKCO2...,{'id': 'chatcmpl-9QCfcmiI5cns2mrDCQXlCi5ClQyZ8...,{'id': 'chatcmpl-9QCfcHMXfab3ag1idZTBmh4QxDdGW...,Scenario,0.1.23
3,33,You are answering questions as if you were a h...,280365,attend church,"Today is November 3, 2020.",woman,slightly conservative,a weak Democrat,Racially I am asian. I am a woman. I am 33 yea...,I never discuss politics with my family or fri...,...,{'id': 'chatcmpl-9QCfcJvcecBdVOfmKaaRE0SYfWJos...,{'id': 'chatcmpl-9QCfcvtRWSoB0YqYmWZIFbXHgui8V...,{'id': 'chatcmpl-9QCfcWz1eaUYpIG07YlmVvHFTxaQG...,{'id': 'chatcmpl-9QCfcttRFDp1MqNd7C8mflifPnvV3...,{'id': 'chatcmpl-9QCfcor9AX7VP1ZrcCNTMabhdYXpX...,{'id': 'chatcmpl-9QCfcW9xKiPULY30SJZm2prY24OEA...,{'id': 'chatcmpl-9QCfcNSP0FHsQncWBFrikA4ZhU69I...,{'id': 'chatcmpl-9QCfc2lScz0O5ymiUYpj4OlbO071z...,Scenario,0.1.23
4,28,You are answering questions as if you were a h...,276123,do not attend church,"Today is November 3, 2020.",woman,liberal,an independent who leans Democratic,Racially I am asian. I am a woman. I am 28 yea...,I never discuss politics with my family or fri...,...,{'id': 'chatcmpl-9QCfcrmJDGicPf1hLnupPIgxvL57k...,{'id': 'chatcmpl-9QCfcenwyvxwNdUUMlZkUpJi9MbRB...,{'id': 'chatcmpl-9QCfciycESMNH0fuEKqTs0elL2MFs...,{'id': 'chatcmpl-9QCfcOpUL8dAWomwomrc33FCNLbNc...,{'id': 'chatcmpl-9QCfcTCQrYzYiqOzaQJZHndLgr4eg...,{'id': 'chatcmpl-9QCfcUnYac3XLXZ8DORm7EHowvb3A...,{'id': 'chatcmpl-9QCfcXlv6t1Ep7K7sqGfmtSqcmpKp...,{'id': 'chatcmpl-9QCfcCW1zymFGGzJMRU6mrqDuDioA...,Scen

## Sense checking responses
We might extend our survey to include questions demonstrating that the LLMs are capable of answering our questions with the given contexts:

In [22]:
from edsl.questions import QuestionFreeText

q_date = QuestionFreeText(question_name="date", question_text="What day is today?")

q_evidence = QuestionFreeText(
    question_name="evidence", question_text="Tell me about current events."
)

survey_check = Survey(questions=[q_date, q_evidence])

results_check = survey_check.by(sample_agents).run()
results_check.select("persona", "date", "evidence").print()

agent.persona,answer.date,answer.evidence
Racially I am asian. I am a woman. I am 33 years old. I am slightly conservative. I am a a weak Democrat. I am not at all in politics. I attend church. I I never discuss politics with my family or friends..,"Today is November 3, 2020.","Current events as of today include the ongoing COVID-19 pandemic, which continues to affect countries worldwide, with many working on vaccine development and distribution strategies. In the United States, the presidential election is a major focus, with citizens heading to the polls to vote for their next president. Additionally, there are various international issues, economic updates, and local news that are also at the forefront of current events. However, as I don't follow politics closely, I can't provide much detail on the political aspects."
Racially I am white. I am a woman. I am 57 years old. I am liberal. I am a an independent who leans Democratic. I am somewhat in politics. I do not attend church. I I never discuss politics with my family or friends..,"Today is November 3, 2020.","Given that today is November 3, 2020, the United States is holding its presidential election. Voters are deciding between re-electing President Donald Trump or electing former Vice President Joe Biden. The election has been heavily influenced by the ongoing COVID-19 pandemic, which has affected campaigning styles and increased the use of mail-in voting. Additionally, issues like the economy, healthcare, racial justice, and climate change have been central themes throughout the campaigns. It's a pivotal moment for the country, and the results are highly anticipated around the world."
Racially I am hispanic. I am a woman. I am 59 years old. I am conservative. I am a a strong Republican. I am not very in politics. I do not attend church. I I never discuss politics with my family or friends..,"Today is November 3, 2020.","As of today, November 3, 2020, the United States is holding its presidential election. Incumbent President Donald Trump, a Republican, is running for re-election against Democratic nominee Joe Biden. The COVID-19 pandemic is a significant issue affecting the election, with many Americans voting by mail or early to avoid crowded polling places. The economy, healthcare, and race relations are also key issues in this election. Additionally, the recent confirmation of Amy Coney Barrett to the U.S. Supreme Court has been a notable event. Globally, countries are grappling with the pandemic, with varying degrees of success in controlling the spread of the virus."
Racially I am asian. I am a woman. I am 28 years old. I am liberal. I am a an independent who leans Democratic. I am not very in politics. I do not attend church. I I never discuss politics with my family or friends..,"Today is November 3, 2020.","As of today, November 3, 2020, the United States is holding its presidential election, where incumbent President Donald Trump, a Republican, is running for re-election against Democratic nominee Joe Biden. This election has seen a significant amount of early voting, partly due to the ongoing COVID-19 pandemic which continues to affect daily life and the economy. In addition to the presidential race, all 435 seats in the House of Representatives and 35 of the 100 seats in the Senate are being contested. Other current events include various countries grappling with the coronavirus as they try to balance public health with economic needs. There are also ongoing discussions about climate change, as extreme weather events continue to highlight the urgency of this global issue."
Racially I am white. I am a man. I am 59 years old. I am slightly liberal. I am a an independent. I am very in politics. I do not attend church. I I like to discuss politics with my family and friends..,"Today is November 3, 2020.","Current events as of today, November 3, 2020, are dominated by the United States presidential election. It's Election Day, and voters across the country are c

Pages 13-14 of the above-mentioned paper https://arxiv.org/pdf/2402.18144.pdf